In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2   ##pretrained cnn model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [2]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, rotation_range=20, zoom_range=0.2, shear_range=0.2, horizontal_flip=True)

In [3]:
val_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [4]:
train_gen = train_datagen.flow_from_directory('Helmet_Dataset_use/train', target_size=(224,224), batch_size=32, class_mode='binary')

Found 2030 images belonging to 2 classes.


In [5]:
test_gen = val_test_datagen.flow_from_directory('Helmet_Dataset_use/test', target_size=(224,224), batch_size=32, class_mode='binary', shuffle=False)

Found 200 images belonging to 2 classes.


In [6]:
val_gen = val_test_datagen.flow_from_directory('Helmet_Dataset_use/validation', target_size=(224,224), batch_size=32, class_mode='binary')

Found 200 images belonging to 2 classes.


In [7]:
model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))  ##ip classifier remove   

In [8]:
model.trainable = False   ##freeze pretrained layers

In [9]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))

In [10]:
base_model.trainable = False 

In [11]:
x = base_model.output  ##feature extract
x = GlobalAveragePooling2D()(x)   ##convert 3d to 1d
x = Dropout(0.5)(x)            ## for prevent overfitting
output = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=output)

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)   ##when accuracy improve epoch will be stop  

In [14]:
history = model.fit(train_gen, validation_data=val_gen, callbacks=[early_stop], epochs=20)

Epoch 1/20


64/64 [==============================] - 223s 3s/step - loss: 0.6966 - accuracy: 0.6251 - val_loss: 0.4002 - val_accuracy: 0.8850
Epoch 2/20
64/64 [==============================] - 207s 3s/step - loss: 0.5177 - accuracy: 0.7488 - val_loss: 0.3125 - val_accuracy: 0.9300
Epoch 3/20
64/64 [==============================] - 210s 3s/step - loss: 0.4711 - accuracy: 0.7877 - val_loss: 0.2695 - val_accuracy: 0.9300
Epoch 4/20
64/64 [==============================] - 219s 3s/step - loss: 0.4083 - accuracy: 0.8246 - val_loss: 0.2371 - val_accuracy: 0.9350
Epoch 5/20
64/64 [==============================] - 186s 3s/step - loss: 0.3857 - accuracy: 0.8217 - val_loss: 0.2205 - val_accuracy: 0.9450
Epoch 6/20
64/64 [==============================] - 190s 3s/step - loss: 0.3807 - accuracy: 0.8394 - val_loss: 0.2075 - val_accuracy: 0.9500
Epoch 7/20
64/64 [==============================] - 190s 3s/step - loss: 0.3658 - accuracy: 0.8379 - val_loss: 0.1910 - val_accuracy: 0.9550
Epoch 8/20


In [15]:
model.evaluate(test_gen)

7/7 [==============================] - 7s 1s/step - loss: 0.2894 - accuracy: 0.8950


[0.2894276976585388, 0.8949999809265137]

In [19]:
img_path =  "nohel__download (1).jpg"  # change to your image path
img = image.load_img(img_path, target_size=(224,224))

img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

img_array = preprocess_input(img_array)
prediction = model.predict(img_array)

if prediction[0][0] > 0.5:
    print("No Helmet")
else:
    print("Helmet")

print("Probability:", prediction[0][0])

1/1 [==============================] - 0s 118ms/step
No Helmet
Probability: 0.9517248


In [20]:
model.save("model.h5")  ##save cnn model

C:\Users\Hp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
